## Task 6

In [42]:
import re

s1 = re.compile("[aeiouóyęą]")
s2 = re.compile("i[aeouęą]")

def syllabicator(word):
    if word in ".,;:'\"-=!?()\\":
        return 0
    
    return len(s1.findall(word)) -len(s2.findall(word))
            
               
               
print(syllabicator("kierunek"))
print(syllabicator("konstantynopolitańczykowianeczka"))



3
12


In [43]:

good_accents = set()

for line in open("../resources/pan-tadeusz.txt"):
    line = re.sub('([-.,!?()])', r' \1 ', line)
    line = re.sub('\s{2,}', ' ', line)
    
    accents = []
    
    for word in line.lower().split():
        syllables = syllabicator(word)
        if syllables > 0:
            accents.append(syllables)
    if(len(accents)>6): #dlugie sa brzydkie
        continue
    
    if sum(accents) == 13:
        good_accents.add(tuple(accents))
        



In [44]:

suffix_ = re.compile("[^aeiouóyęą]*[aeiouóyęą][^aeiouóyęą]*[aeiouóyęą]")
def get_suffix(word):
    match = suffix_.search(word[::-1])
    if match:
        return match.group(0)[::-1]
    else:
        return word

print(get_suffix("konstantynopolitańczykowianeczka"))
print(get_suffix("pplklpklkjplkj"))

eczka
pplklpklkjplkj


In [45]:
rhymes = {} #from suffix to list of words
bigrams = {} #from snd in bigrams to list of n_syllables to words
unigrams = [{} for _ in range(9)]

for line in open("../poleval_corpus/poleval_2grams.txt"):
    count, prev, word = line.split()
    count = int(count)
    
    word_syllables = syllabicator(word)
    prev_syllables = syllabicator(prev)
    
    if word_syllables <= 0 or prev_syllables <= 0 or \
       word_syllables >= 9 or prev_syllables >= 9:
        continue
    
    suffix = get_suffix(word)
    
    if suffix not in rhymes:
        rhymes[suffix] = [set() for _ in range(9)]
    
    rhymes[suffix][word_syllables].add(word)
    
    unigrams[word_syllables][word] = unigrams[word_syllables].get(word, 0) + count
    unigrams[prev_syllables][prev] = unigrams[prev_syllables].get(prev, 0) + count
    
    if word not in bigrams:
        bigrams[word] = [{} for _ in range(9)]
        
    bigrams[word][prev_syllables][prev] = count
    


In [61]:
#wylosuj rozkład rymów dla dwóch linijek

# wylosuj ostatnie slowo
# wylosuj do niego rym

# losuj słowa

import random

good_accents = list(good_accents)


def make_line(accents, prev):
    
    res = [prev]
    for accent in reversed(accents):
        if prev in bigrams and bigrams[prev][accent]:
            word = random.choices(list(bigrams[prev][accent].keys()), weights=list(bigrams[prev][accent].values()))[0]
        else:
            word = random.choices(list(unigrams[accent].keys()), weights=list(unigrams[accent].values()))[0]
        res.append(word)
        prev = word
    return " ".join(reversed(res))

def get_poem():
    
    good = False
    while not good:
        good = True
        
        second_line_accents = random.choice(good_accents)
        first_line_accents = random.choice(good_accents)
        
        #wylosuj pierwsze słowo
        
        n_syllables = -1
        while n_syllables != first_line_accents[-1]:
            first_word = random.choice(list(bigrams.keys()))
            n_syllables = syllabicator(first_word)
        
        first_suffix = get_suffix(first_word)
        
        if not rhymes[first_suffix][n_syllables]:
            good = False
            continue
            
        second_word = random.choice(list(rhymes[first_suffix][n_syllables]))
        
        first_line = make_line(first_line_accents[:-1], first_word)
        second_line = make_line(second_line_accents[:-1], second_word)
        
    return [first_line, second_line]


get_poem()

['swój wkład dziękuję bardzo długim klinowatym',
 'mieszkaniową a które się o groszkowatym']

In [1]:
get_poem()

NameError: name 'get_poem' is not defined